NAME: Mohammadreza Ahmadi Teshnizi
Student number: 98170646
Colab link:https://colab.research.google.com/drive/1BqdjoZS17_3q4s_wiAwFQopFVKNbvX3B?usp=sharing

In [ ]:
!pip install pandas
!pip install numpy
!pip install sklearn

In [13]:
import pandas as pd
import gdown

url = 'https://drive.google.com/uc?id=1BRhlDz-JASvfEljf_XxrL-ZH4gUvOg0v'
output =  'data-train.csv'

gdown.download(url, output, quiet=False)
data = pd.read_csv('data-train.csv')

print(data.head())

Downloading...
From: https://drive.google.com/uc?id=1BRhlDz-JASvfEljf_XxrL-ZH4gUvOg0v
To: /content/data-train.csv
100%|██████████| 8.48M/8.48M [00:00<00:00, 231MB/s]


   PhraseId  SentenceId                                             Phrase  \
0         1           1  A series of escapades demonstrating the adage ...   
1         2           1  A series of escapades demonstrating the adage ...   
2         3           1                                           A series   
3         4           1                                                  A   
4         5           1                                             series   

   Sentiment  
0          1  
1          2  
2          2  
3          2  
4          2  


In [15]:

import pandas as pd
from sklearn.model_selection import train_test_split

train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Training data: {train_data.shape}")
print(f"Validation data: {valid_data.shape}")
print(f"Testing data: {test_data.shape}")

Training data: (124038, 4)
Validation data: (15505, 4)
Testing data: (15505, 4)


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

stemmer = PorterStemmer()

def preprocess_data_basic(data):
    data = data.lower()
    data = re.sub(r'[^\w\s]', '', data)
    data = ' '.join(stemmer.stem(word) for word in data.split() if word not in stop_words)
    return data

train_data['Phrase'] = train_data['Phrase'].apply(preprocess_data_basic)
vectorizer = TfidfVectorizer()
train_features = vectorizer.fit_transform(train_data['Phrase'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
from gensim.models import Word2Vec
import numpy as np

def preprocess_data_w2v(data):
    data = data.lower()
    data = re.sub(r'[^\w\s]', '', data).split()
    data = [word for word in data if word not in stop_words]
    return data

train_data_w2v = train_data['Phrase'].apply(preprocess_data_w2v)

model = Word2Vec(sentences=train_data_w2v, vector_size=100, window=5, min_count=1, workers=4)

def phrase_vector(phrase):
    phrase_vec = []
    for word in phrase:
        try:
            phrase_vec.append(model.wv[word])
        except KeyError:
            pass
    if phrase_vec:
        return sum(phrase_vec)/len(phrase_vec)
    else:
        return np.zeros(model.vector_size)

train_features_w2v = np.array([phrase_vector(phrase) for phrase in train_data_w2v])

In [22]:
sentiment_counts = train_data['Sentiment'].value_counts()
print(sentiment_counts)

2    63330
3    26157
1    21622
4     7288
0     5641
Name: Sentiment, dtype: int64


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10, 100]}

# Approach 1: Basic Methods
grid = GridSearchCV(LogisticRegression(max_iter=100000), param_grid, refit=True, verbose=3)
grid.fit(train_features, train_data['Sentiment'])

valid_predictions_basic = grid.predict(vectorizer.transform(valid_data['Phrase']))

print("Approach 1: Basic Methods")
print(classification_report(valid_data['Sentiment'], valid_predictions_basic))

# Approach 2: Word2Vec
grid = GridSearchCV(LogisticRegression(max_iter=100000), param_grid, refit=True, verbose=3)
grid.fit(train_features_w2v, train_data['Sentiment'])

valid_predictions_w2v = grid.predict([phrase_vector(phrase) for phrase in valid_data['Phrase'].apply(preprocess_data_w2v)])

print("\nApproach 2: Word2Vec")
print(classification_report(valid_data['Sentiment'], valid_predictions_w2v))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END .............................C=0.1;, score=0.565 total time=   9.8s
[CV 2/5] END .............................C=0.1;, score=0.564 total time=   8.7s
[CV 3/5] END .............................C=0.1;, score=0.562 total time=   9.5s
[CV 4/5] END .............................C=0.1;, score=0.561 total time=   8.6s
[CV 5/5] END .............................C=0.1;, score=0.563 total time=  10.2s
[CV 1/5] END ...............................C=1;, score=0.624 total time=  25.9s
[CV 2/5] END ...............................C=1;, score=0.626 total time=  27.9s
[CV 3/5] END ...............................C=1;, score=0.620 total time=  29.6s
[CV 4/5] END ...............................C=1;, score=0.624 total time=  29.1s
[CV 5/5] END ...............................C=1;, score=0.627 total time=  27.0s
[CV 1/5] END ..............................C=10;, score=0.635 total time= 1.1min
[CV 2/5] END ..............................C=10;,